In [ ]:
import os
import re
import altair as alt
import pandas as pd

colordomain = ['Solution', 'None', 'UND']
colorrange = ['#4DAF4A', '#E41A1C', '#377EB8']

def isNumber(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

def getResults(shape, getInProgress=False, getCancelled=False):
    for root, _, fs in os.walk(shape):
        files =  [os.path.join(root, f) for f in fs]
        break
    results = []
    inProgressCounter = 0
    cancelledCounter = 0
    for path in files:
        with open(path) as f:
            lines = f.readlines()[-7]
        result = lines.strip()
        if result == '=========================================================':
            inProgressCounter += 1
            if not getInProgress:
                continue
            result = 'In progress'
        if result.startswith('Job submitted date = '):
            cancelledCounter += 1
            if not getCancelled:
                continue
            result = "Cancelled"
        if result == '---------------' or result == "Job output ends":
            result = 'Error'
        category = result
            
        if not result in ['Cancelled', 'UND', 'None', 'Error', 'TIMEOUT', 'In progress']:
            category = 'Solution'
            ratio = 1.0
        else:
            ratio = 0.0
            
        if isNumber(result):
            ratio = float(result)
            category = 'UND'

        m = re.search('([0-9]+)t_([0-9]+)c-([0-9]+).out', path)
        m.group(0)
        results.append({
            'nCubeTypes': int(m.group(1)),
            'nColors': int(m.group(2)),
            'jobID': int(m.group(3)),
            'result': result,
            'category': category,
            'ratio': ratio
        })
    df = pd.DataFrame(results)
    df.to_csv(shape+'.csv')
    print("Found {} in progress and {} cancelled".format(inProgressCounter, cancelledCounter))
    return df

def plotData(df):
    base = alt.Chart(df).transform_calculate(
        url='https://akodiat.github.io/polycubes?rule=' + alt.datum.result
    )
    return base.mark_rect().encode(
        alt.X('nCubeTypes:O', title='# of species'),
        alt.Y('nColors:O', title='# of colors', sort='-y'),
        color=alt.Color('category', scale=alt.Scale(domain=colordomain, range=colorrange)),
        href='url:N'
    ) + base.mark_circle(color='black', opacity=0.5).encode(
        alt.X('nCubeTypes:O', title='# of species'),
        alt.Y('nColors:O', title='# of colors', sort='-y'),
        size=alt.Size('ratio:Q'),
        tooltip=['result', 'nCubeTypes', 'nColors', 'jobID', 'ratio'],
        href='url:N'
    )

In [ ]:
plotData(getResults('square')).properties(
    title='Square',
    width=200, height=200
)

In [ ]:
plotData(getResults('2rectangles')).properties(
    title='Multifarious rectangles'
)

In [ ]:
plotData(getResults('cube')).properties(
    title='Cube'
)

In [ ]:
plotData(getResults('cross', True, False)).properties(
    title='3D cross'
)

In [ ]:
plotData(getResults('chair')).properties(
    title='Chair'
)

In [ ]:
def getInProgressOverCompl(name, maxNC, maxNT):
    return getResults(name, getInProgress=True).query(
        "category == 'In progress' and (nCubeTypes > {} or nColors > {})".format(
            maxNC, maxNT
    ))['jobID'].values.tolist()

In [ ]:
print(' '.join(["{}".format(id) for id in getInProgressOverCompl('cube', 7,8)]))

In [ ]:
def getInProgress(name):
    return getResults(name, getInProgress=True).query("category == 'In progress'")['jobID'].values.tolist()

In [ ]:
print(' '.join(["{}".format(id) for id in getInProgress('chair')]))